In [1]:
import os
import sys
sys.path.append('D:\\Users\\Hegxiten\\workspace\\Rutgers_Railway_security_research\\OOD_Train')

from datetime import datetime, timedelta
import numpy as np
import random
from infrastructure import Track, Block, BigBlock
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint
from system import System

import networkx as nx


In [2]:
class Train():
    def __init__(self, idx, rank, system, init_time, init_direction, max_sp, max_acc, max_dcc):        
        ((_prev_sigpoint,_prev_sigport),(_curr_sigpoint, _prev_sigport)) = init_direction
        self._curr_direction = init_direction
        self._curr_MP = self.curr_sigpoint.MP
        self._curr_track = None
        self.train_idx = idx
        self.rank = rank
        self.system = system
        self.max_speed = max_sp
        self.max_acc = max_acc
        self.max_dcc = max_dcc
        self._curr_speed = 0
        
        # self.blk_interval = system.block_intervals
        self.time_pos_list = []        
        # self.status = 1
    
    @property
    def stopped(self):
        if self.curr_speed == 0 and self.curr_sig.aspect.color == 'r':
            return True
        else:
            return False
    
    @property
    def curr_MP(self):
        return self._curr_MP

    @curr_MP.setter
    def curr_MP(self, new_MP):
        if self.prev_sigpoint and self.curr_sigpoint:
            _MP_pair = (self.prev_sigpoint.MP, self.curr_sigpoint.MP)
            if min(_MP_pair) < new_MP < max(_MP_pair): 
                self._curr_MP = new_MP
            elif self.curr_sig.permit_track:
                self.cross_sigpoint(self.curr_sigpoint, self._curr_MP, new_MP)
                self._curr_MP = new_MP
            else:
                self.cross_sigpoint(self.curr_sigpoint, self._curr_MP, new_MP, terminate=True)
                self._curr_MP = self.curr_sigpoint.MP
        elif not self.prev_sigpoint:
            assert self.curr_sigpoint
            self.cross_sigpoint(self.curr_sigpoint, self._curr_MP, new_MP, initiate=True)
            self._curr_MP = new_MP
        else: 
            raise ValueError('Setting MP Failed: current MP: {}, new MP: {}, current track: {}, current aspect: {}, permit track: {}'\
                .format(self._curr_MP, new_MP, self._curr_track, self.curr_sig.aspect, self.curr_sig.permit_track))
        self.time_pos_list.append([self.system.sys_time + self.system.refresh_time, self._curr_MP])

    @property
    def curr_direction(self):
        return self._curr_direction

    @curr_direction.setter
    def curr_direction(self,new_direction):
        assert isinstance(new_direction, tuple) and len(new_direction) == 2
        self._curr_direction = new_direction

    @property
    def curr_track(self):
        (_prev_point, _prev_port) = self._curr_direction[0]
        (_next_point, _next_port) = self._curr_direction[1]
        assert _next_point.track_by_port.get(_next_port) == _next_point.track_by_port.get(_next_port)
        self._curr_track = _next_point.track_by_port.get(_next_port)
        return self._curr_track
    
    @property
    def curr_sigpoint(self):
        return self.curr_direction[1][0]
    
    @property
    def curr_sigport(self):
        return self.curr_direction[1][1]
    
    @property
    def prev_sigpoint(self):
        return self.curr_direction[0][0]    
    
    @property
    def curr_sig(self):
        return self.curr_sigpoint.signal_by_port[self.curr_sigport]

    @property
    def curr_acc(self):
        if not self.stopped:
            if self.curr_target_speed > self.curr_speed:
                return self.max_acc
            elif self.curr_target_speed == self.curr_speed:
                return 0
            else:
                return -self.max_dcc
        else:
            return 0
                # braking distance有可能要写到acc的setter里 

    @property
    def curr_speed(self):
        return self._curr_speed

    @curr_speed.setter
    def curr_speed(self, new_speed):
        _old_speed = self._curr_speed
        if new_speed * _old_speed < 0:  
            self._curr_speed = 0
        else: 
            self._curr_speed = new_speed
        # 这里判断braking distance有可能要写到acc里
        if self.curr_brake_distance > self.curr_dis_to_curr_sig:
            self._curr_speed = _old_speed
        assert self.curr_brake_distance <= self.curr_dis_to_curr_sig

    @property
    def curr_target_speed(self):
        return self.curr_sig.aspect.target_speed

    @property
    def curr_brake_distance(self):
        if abs(self.curr_target_speed) < abs(self.curr_speed):
            return abs(self.curr_target_speed ** 2 - self.curr_speed ** 2) / self.curr_acc
        else:
            return 0.0

    @property
    def curr_dis_to_curr_sig(self):
        return abs(self.curr_sigpoint.MP - self.curr_MP)

    def __repr__(self):
        return 'train index {}, current direction {}'\
            .format(self.train_idx, self.curr_direction)

    def __lt__(self, othertrain):
        if self.curr_MP > othertrain.curr_MP:
            return True
        elif self.curr_MP < othertrain.curr_MP:
            return False
        # when the MP is the same:
        elif self.max_speed > othertrain.max_speed:
            return True
        elif self.max_speed < othertrain.max_speed:
            return False
        # elif self.rank < othertrain.rank:
        #     return False
        else:
            return True

    def cross_sigpoint(self, sigpoint, curr_MP, new_MP, initiate=False, terminate=False):
        assert self.curr_sig.route in sigpoint.current_routes
        assert min(curr_MP, new_MP) <= sigpoint.MP <= max(curr_MP, new_MP)
        assert not self.stopped
        if self.curr_speed != 0:
            timestamp = self.system.sys_time + abs(curr_MP - sigpoint.MP)/abs(self.curr_speed)
        else:
            timestamp = self.system.sys_time
        self.time_pos_list.append([timestamp, sigpoint.MP])

        if initiate:
            assert isinstance(sigpoint, ControlPoint)
            assert len(self.curr_sig.permit_track.train) == 0
            print('train {} moved into track {}'.format(self, self.curr_sig.permit_track))
            self.curr_sig.permit_track.train.append(self)
            self.curr_direction = ((sigpoint, self.curr_sig.route[1]), (self.curr_sig.next_enroute_sigpoint, self.curr_sig.next_enroute_sigpoint_port))
            sigpoint.close_route(self.curr_sig.route)
        elif terminate:
            assert isinstance(sigpoint, ControlPoint)
            self.curr_track.train.remove(self)
            self.curr_direction = ((sigpoint, self.curr_sig.route[1]), (None, None))
            sigpoint.close_route(self.curr_sig.route)
        elif not initiate and not terminate:
            assert len(self.curr_sig.permit_track.train) == 0
            self.curr_sig.permit_track.train.append(self)
            self.curr_track.train.remove(self)
            self.curr_direction = ((sigpoint, self.curr_sig.route[1]), (self.curr_sig.next_enroute_sigpoint, self.curr_sig.next_enroute_sigpoint_port))
            if isinstance(sigpoint, ControlPoint):
                sigpoint.close_route(self.curr_sig.route)
        else:
            raise ValueError('train {} crossing signalpoint {} failed unexpectedly'\
                .format(self, sigpoint))
    
    def update_acc(self):
        if not self.stopped:
            delta_s = self.curr_speed * self.system.refresh_time + 0.5 * self.curr_acc * self.system.refresh_time ** 2
            self.curr_speed = self.curr_speed + self.curr_acc * self.system.refresh_time
            self.curr_MP += delta_s
            self.time_pos_list.append([self.system.sys_time+self.system.refresh_time, self.curr_MP])
            

In [3]:
sim_init_time = datetime.strptime('2018-01-10 10:00:00', "%Y-%m-%d %H:%M:%S")
sim_term_time = datetime.strptime('2018-01-10 15:30:00', "%Y-%m-%d %H:%M:%S")
sp_container = [random.uniform(0.01, 0.02) for i in range(20)]
acc_container = [random.uniform(2.78e-05*0.85, 2.78e-05*1.15) for i in range(20)]
dcc_container = [random.uniform(2.78e-05*0.85, 2.78e-05*1.15) for i in range(20)]
headway = 200 * random.random() + 400
sys = System(sim_init_time, sp_container, acc_container, dcc_container,
             dos_period=['2018-01-10 11:30:00', '2018-01-10 12:30:00'],  
             headway=headway, 
             tracks=[1,1,1,4,1,1,3,1,1,1], 
             dos_pos=-1)
k165 = Train(idx=sys.train_num, 
                  rank=sys.train_num, 
                  system=sys, 
                  init_time=sys.sys_time, 
                  init_direction=((None,None),(sys.control_points[0],0)), 
                  max_sp=sys.sp_container[sys.train_num % len(sys.sp_container)], 
                  max_acc=sys.acc_container[sys.train_num % len(sys.acc_container)], 
                  max_dcc=sys.dcc_container[sys.train_num % len(sys.dcc_container)])

In [4]:
for i in range(36000):
    print(k165.stopped, round(k165.curr_MP,5), round(k165.curr_speed*3600,5)\
          , round(k165.curr_sig.aspect.target_speed*3600,2), 'current acc',\
         k165.curr_acc)
    k165.update_acc()
    sys.sys_time+=sys.refresh_time


True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0

True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0

True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0

True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0 current acc 0
True 0.0 0 0.0

KeyboardInterrupt: 

In [5]:
k165.curr_MP

10.583385191701906

In [9]:
for n in sys.control_points:
    n.open_route((0,1))
for n in sys.signal_points:
    print(n.signal_by_port[0].aspect)
    

route (0, 1) for ControlPoint0 already opened
route (0, 1) for ControlPoint3 already opened
route (0, 1) for ControlPoint4 already opened
route (0, 1) for ControlPoint6 already opened
route (0, 1) for ControlPoint7 already opened
route (0, 1) for ControlPoint10 already opened
Aspect: g, 	 route (0, 1), target speed 72.0 mph
Aspect: g, 	 route (0, 1), target speed 72.0 mph
Aspect: g, 	 route (0, 1), target speed 72.0 mph
Aspect: g, 	 route (0, 1), target speed 72.0 mph
Aspect: g, 	 route (0, 1), target speed 72.0 mph
Aspect: g, 	 route (0, 1), target speed 72.0 mph
Aspect: g, 	 route (0, 1), target speed 72.0 mph
Aspect: g, 	 route (0, 1), target speed 72.0 mph
Aspect: g, 	 route (0, 1), target speed 72.0 mph
Aspect: g, 	 route (0, 1), target speed 72.0 mph
Aspect: g, 	 route (0, 1), target speed 72.0 mph


In [26]:
k165.__dict__

{'_curr_direction': ((None, None), (ControlPoint0, 0)),
 '_curr_MP': 0.0,
 '_curr_track': None,
 'train_idx': 0,
 'rank': 0,
 'system': <system.System at 0x2b6b025dbe0>,
 'max_speed': 0.018641396086126354,
 'max_acc': 2.8223590863411127e-05,
 'max_dcc': 3.187306687071804e-05,
 '_curr_speed': 0,
 'time_pos_list': []}

0.0 0 72.0 current acc 0
train train index 0, current direction ((None, None), (ControlPoint0, 0)) moved into track Track MP: 0.0 to MP: 5.0 idx: 0
route (0, 1) for ControlPoint0 is closed
0.0 0.1 40.0 current acc 0
0.0 0.2 40.0 current acc 0
0.0 0.3 40.0 current acc 0
0.0 0.41 40.0 current acc 0
0.0 0.51 40.0 current acc 0
0.0 0.61 40.0 current acc 0
0.0 0.71 40.0 current acc 0
0.0 0.81 40.0 current acc 0
0.0 0.91 40.0 current acc 0
0.0 1.02 40.0 current acc 0
0.0 1.12 40.0 current acc 0
0.0 1.22 40.0 current acc 0
0.0 1.32 40.0 current acc 0
0.0 1.42 40.0 current acc 0
0.0 1.52 40.0 current acc 0
0.0 1.63 40.0 current acc 0
0.0 1.73 40.0 current acc 0
0.0 1.83 40.0 current acc 0
0.0 1.93 40.0 current acc 0
0.0 2.03 40.0 current acc 0
0.0 2.13 40.0 current acc 0
0.0 2.24 40.0 current acc 0
0.0 2.34 40.0 current acc 0
0.0 2.44 40.0 current acc 0
0.0 2.54 40.0 current acc 0
0.0 2.64 40.0 current acc 0
0.0 2.74 40.0 current acc 0
0.0 2.84 40.0 current acc 0
0.0 2.95 40.0 current acc 0
0.

2.0 38.41 40.0 current acc 0
2.0 38.51 40.0 current acc 0
2.0 38.61 40.0 current acc 0
2.0 38.71 40.0 current acc 0
2.1 38.81 40.0 current acc 0
2.1 38.91 40.0 current acc 0
2.1 39.02 40.0 current acc 0
2.1 39.12 40.0 current acc 0
2.1 39.22 40.0 current acc 0
2.1 39.32 40.0 current acc 0
2.1 39.42 40.0 current acc 0
2.1 39.52 40.0 current acc 0
2.1 39.63 40.0 current acc 0
2.2 39.73 40.0 current acc 0
2.2 39.83 40.0 current acc 0
2.2 39.93 40.0 current acc 0
2.2 40.03 40.0 current acc 0
2.2 40.15 40.0 current acc 0
2.2 40.26 40.0 current acc 0
2.2 40.38 40.0 current acc 0
2.2 40.49 40.0 current acc 0
2.2 40.61 40.0 current acc 0
2.3 40.72 40.0 current acc 0
2.3 40.84 40.0 current acc 0
2.3 40.95 40.0 current acc 0
2.3 41.07 40.0 current acc 0
2.3 41.18 40.0 current acc 0
2.3 41.29 40.0 current acc 0
2.3 41.41 40.0 current acc 0
2.3 41.52 40.0 current acc 0
2.3 41.64 40.0 current acc 0
2.4 41.75 40.0 current acc 0
2.4 41.87 40.0 current acc 0
2.4 41.98 40.0 current acc 0
2.4 42.1 40.0 

AssertionError: 

In [20]:
k165.curr_speed = 50

AssertionError: 